### Source: https://data.police.uk/docs/

In [ ]:
import pandas as pd
import os
import requests as r

### Specify path of crimes folder

In [ ]:
path = 'crimes.xlsx'

### Read and concatenate data

In [ ]:
def load_data(path):
    
    
    df = pd.DataFrame()
    
    # Iterate through all folders in path and read
    for root, subdirectories, files in os.walk(path):
        for file in files:
            temp = pd.read_csv(os.path.join(root, file))
            df = pd.concat([df, pd.DataFrame.from_records(temp)])
    
    return df

In [ ]:
df = load_data(path)

### Convert longitude and latitude to Postal Codes

In [ ]:
# Data cleaning
df['Year'] = df['Month'].str.split('-').str[0]
df['Month'] = df['Month'].str.split('-').str[1]

# Drop empty longitude and latitudes
df.dropna(subset = ['Longitude', 'Latitude'], inplace = True)

# Integer encode crime type
c = df['Crime type'].astype('category')
df['Crime type'] = df['Crime type'].astype('category').cat.codes

# Create mapping
map_cat = dict(enumerate(c.cat.categories))

# Create postal code column
df['Postal Code'] = 0

# Reset index
df.reset_index(drop = True, inplace = True)

# Use API to convert lat and long to postal code
for i in range(len(df)):
    latitude = df['Latitude'][i]
    longitude = df['Longitude'][i]
    response = r.get('https://findthatpostcode.uk/points/' + str(latitude) + '%2C' + str(longitude) + '.json')
    postal_code = response.json()['data']['relationships']['nearest_postcode']['data']['id']
    df.loc[i, 'Postal Code'] = postal_code

### Convert to pivot

In [8]:
df_pivot.to_excel('C:/Users/Alexander.Lee/OneDrive - Community Solutions/Desktop/postal_codes.xlsx', sheet_name = 'sheet1', index = False)